In [ ]:
import numpy as np
import csv
import pandas as pd
from scipy import signal
from scipy.signal import butter, lfilter, freqz
from matplotlib import pyplot as plt
import math as m


In [ ]:
def plotdata(data,title = 'sEMG',static_idx = -1,spec_use = False):
    start_pt = 0
    x_axis_value = range(start_pt,len(data))
    plt.plot(x_axis_value,data,color='r')
    
    plt.ylabel("amplitude")
    if title == 'muscle activation ':
        plt.title(title+r'$\psi$')
        plt.xlabel("number of the 150ms time window")
    else:
        plt.title(title)
        plt.xlabel("time(ms)")
    if static_idx != -1:
        
        static_mean = sum(data[:static_idx])/static_idx
        std = np.std(data[:static_idx])
        plt.axhline(static_mean,color = 'g',label = 'static_mean')
        #plt.axhline(3*static_mean,color = 'b')
        plt.axhline(static_mean+3*std,color = 'b',label = 'mean+3*std')
        plt.legend(loc = "best", fontsize=10)
    if spec_use :
        plt.savefig(title,dpi = 200)
    #try to visualize threshold
    plt.show()
    if static_idx != -1:
        return static_mean,std 
    else:
        return 0,0
    
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs  # nyquist frequency
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

#notch filter
def notch_filter(data, f0, Q, fs):
    b, a = signal.iirnotch(f0, Q, fs)
    y = lfilter(b, a, data)
    return y

#lowpass filter
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a
def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

#highpass filter
def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a
def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

#Full wave rectification
#calculate absolute value

def FilterData(data, sampling_rate):
    #not1 = notch_filter(data, 60, 30, sampling_rate)
    #plotdata(not1,'not1')
    #high1 = butter_highpass_filter(data, 10, sampling_rate, order=6)
    #plotdata(high1,'high1')
    high1 = butter_bandpass_filter(data,20,400,sampling_rate,order = 5)
    #rec1 = abs(high1)
    #plotdata(rec1,'rec1')
    #lpass1 = butter_lowpass_filter(rec1, 499, sampling_rate, order = 6)
    #plotdata(lpass1,'lpass1')
    win = signal.windows.hann(20)
    filtered = signal.convolve(high1, win, mode='same') / sum(win)
    rec = abs(filtered)
    return rec


In [ ]:
count = 0
ch = []
path = './subject2/test2/second.txt'

# for txt format
with open(path) as f:
    for line in f.readlines():
        count += 1
        if count >= 7 and len(line) > 5:
            s = line.split('\t')    # split channel data
            k = s[3].split('\n')
            s[3] = k[0]             # remove '\n'
            s = list(map(int, s))   # convert string to int
            ch.append(list(s))
ch=np.transpose(ch) 
print(np.shape(ch))
f.close

# for csv format
# csvReader = csv.reader(open(path2,'r'), delimiter=',')
# data  = np.array(list(csvReader))
# print(data.shape)
# ch = []
# for i in range(8):
#     ch.append(list(data[1:,i].astype(float)))
# print(len(ch),len(ch[0]))

In [ ]:
#filter_period 調整多久做一次濾波處理
#filter_period = len(data_in_float[0]) 表示全部一起做。

#filter_period = len(data_in_float[0])
filter_period = 150
fdata = []
segmented_data= []

for j in range(3):
    temp_fdata = []
    for i in range( int(len(ch[0])/filter_period)):
        cur_window = FilterData(ch[j+1][i*filter_period : (i+1)*filter_period], 1000)
        temp_fdata.extend(cur_window)
        
    fdata.append(temp_fdata)

    
print(len(fdata[0]))
print(type(fdata))



ch1 = (list( data[1:,0].astype(float) ))
print(len(ch1),ch1[0:2],type(ch1))
ch1_fdata = FilterData(ch1 ,1000)




In [ ]:
for i in range(3):
    plotdata(ch[i][:],'raw'+str(i))
    plotdata(fdata[i][:] ,'f'+str(i))

## rest_data

In [ ]:
ch_r=[]
count=0
path_rest = './subject2/test2/rest.txt'

with open(path_rest) as f:
    for line in f.readlines():
        count+=1
        if(count>=8 and len(line)>5):
            s = line.split('\t')
            k=s[3].split('\n')
            s[3]=k[0]
            s = list(map(int, s))
            ch_r.append(list(s))
ch_r=np.transpose(ch_r) 
print(np.shape(ch_r))
f.close

filter_period = 150
fdata_r = []
segmented_data= []

for j in range(3):
    temp_fdata = []
    for i in range( int(len(ch_r[0])/filter_period)):
        cur_window = FilterData(ch_r[j+1][i*filter_period:(i+1)*filter_period],1000)
        temp_fdata.extend(cur_window)
        
    fdata_r.append(temp_fdata)
for i in range(3):
    plotdata(ch_r[i][:],'raw'+str(i))
    plotdata(fdata_r[i][:] ,'f'+str(i))